In [1]:
import numpy as np
from pandas import DataFrame
import pandas as pd
from pandas.io.json import json_normalize
import xml.etree.ElementTree as ET
import requests
import json
#from flask import Flask
#from flask_restful import Api,Resource,reqparse
#import pyzillow

In [2]:
zwsid = open('../../certs/ZWSID','r')

In [3]:
zwsid = zwsid.read()

In [4]:
def query_builder(endpoint,query):
    string = '?'
    for v in query:
        string+='&{}={}'.format(v,query[v])
    #string = '%'.join(string.split(' '))
    return endpoint+string

#final_endpoint that works 
http://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id=X1-ZWz1gptphnw3kb_9wsfl&address=1551 Shammrock Dr&citystatezip=Gardendale, AL 35071

In [22]:
address='4613 Fieldstown Way'
city_state_zip = 'Gardendale, Al 35071'

deep_search_endpoint = 'http://www.zillow.com/webservice/GetDeepSearchResults.htm'
deep_query = {'address':address,'citystatezip':city_state_zip,'zws-id':zwsid}


In [23]:
#test the api endpoint
query_builder(deep_search_endpoint,deep_query)

'http://www.zillow.com/webservice/GetDeepSearchResults.htm?&zws-id=X1-ZWz1gptphnw3kb_9wsfl&citystatezip=Gardendale, Al 35071&address=4613 Fieldstown Way'

In [24]:
# Get the feed
def deepSearch_api(endPoint,query):
    r = requests.get(query_builder(endPoint,query))
    r.text
    return_xml = ET.fromstring(r.text)
    resp = return_xml.find('response')
    d={}
    for x in resp.iter():
        for child in x.getiterator():
            d[child.tag] =child.text
    return d

In [25]:
deep_results=deepSearch_api(deep_search_endpoint,deep_query)

In [26]:
deep_results

{'FIPScounty': None,
 'address': None,
 'amount': '319727',
 'bathrooms': '2.5',
 'bedrooms': '4',
 'city': 'Gardendale',
 'comparables': 'http://www.zillow.com/homes/comps/2091548116_zpid/',
 'finishedSqFt': '2704',
 'forSale': 'http://www.zillow.com/gardendale-al/',
 'forSaleByOwner': 'http://www.zillow.com/gardendale-al/fsbo/',
 'graphsanddata': 'http://www.zillow.com/homedetails/4613-Fieldstown-Way-Gardendale-AL-35071/174774784_zpid/#charts-and-data',
 'high': '335713',
 'homedetails': 'https://www.zillow.com/homedetails/4613-Fieldstown-Way-Gardendale-AL-35071/2091548116_zpid/',
 'last-updated': '10/25/2018',
 'lastSoldDate': '10/07/2015',
 'lastSoldPrice': '275000',
 'latitude': '33.640965',
 'links': None,
 'localRealEstate': None,
 'longitude': '-86.843863',
 'low': '287754',
 'mapthishome': 'http://www.zillow.com/homes/2091548116_zpid/',
 'oneWeekChange': None,
 'overview': 'http://www.zillow.com/local-info/AL-Gardendale/r_31697/',
 'percentile': '0',
 'region': None,
 'respons

In [27]:
zpid=deep_results['zpid']

In [28]:
def deepComps_api(endPoint,query):
    r = requests.get(query_builder(comps_endpoint,comps_query))
    r.text
    return_xml = ET.fromstring(r.text)
    i=[]
    resp = return_xml.find('response')
    for x in resp.iter('comp'):
        d={}
        i.append(d)
        for child in x.getiterator():
            d[child.tag]=child.text
    df = pd.concat([json_normalize(x) for x in i],join='outer')
    df.dropna(1,inplace=True)
    df.reset_index(inplace=True)
    df.drop('index',1,inplace=True)
    return df

In [59]:
comps_deep_endpoint = 'http://www.zillow.com/webservice/GetDeepComps.htm'
comps_query = {'zpid':zpid,'count':23,'zws-id':zwsid}

In [30]:
comps =deepComps_api(comps_endpoint,comps_query)

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  if sys.path[0] == '':


In [33]:
df =comps[comps['percentile']>'70'].sort_values('percentile',ascending=False)

In [38]:
comps_endpoint = 'http://www.zillow.com/webservice/GetComps.htm'
comps_query = {'zpid':zpid,'count':23,'zws-id':zwsid}

In [60]:
df =deepComps_api(comps_deep_endpoint,comps_query)
#/Comps:comps/response/properties/comparables

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  if sys.path[0] == '':


In [61]:
df

,city,comparables,forSale,forSaleByOwner,homedetails,last-updated,latitude,longitude,mapthishome,overview,percentile,state,street,zindexValue,zipcode,zpid
0,MOUNT OLIVE,http://www.zillow.com/homes/comps/911903_zpid/,http://www.zillow.com/mount-olive-al/,http://www.zillow.com/mount-olive-al/fsbo/,https://www.zillow.com/homedetails/699-Powers-...,10/25/2018,33.666142,-86.849879,http://www.zillow.com/homes/911903_zpid/,http://www.zillow.com/local-info/AL-Mount-Oliv...,93,AL,699 Powers Cir,"146,600",35117,911903
1,MOUNT OLIVE,http://www.zillow.com/homes/comps/911915_zpid/,http://www.zillow.com/mount-olive-al/,http://www.zillow.com/mount-olive-al/fsbo/,https://www.zillow.com/homedetails/5307-Roland...,10/25/2018,33.664846,-86.848247,http://www.zillow.com/homes/911915_zpid/,http://www.zillow.com/local-info/AL-Mount-Oliv...,91,AL,5307 Roland Cir,"146,600",35117,911915
2,MOUNT OLIVE,http://www.zillow.com/homes/comps/911993_zpid/,http://www.zillow.com/mount-olive-al/,http://www.zillow.com/mount-olive-al/fsbo/,https://www.zillow.com/homedetails/5301-Pinecr...,10/25/2018,33.661584,-86.859169,http://www.zillow.com/homes/911993_zpid/,http://www.zillow.com/local-info/AL-Mount-Oliv...,76,AL,5301 Pinecrest Dr,"146,600",35117,911993
3,MOUNT OLIVE,http://www.zillow.com/homes/comps/912087_zpid/,http://www.zillow.com/mount-olive-al/,http://www.zillow.com/mount-olive-al/fsbo/,https://www.zillow.com/homedetails/530-Gary-Dr...,10/25/2018,33.659124,-86.847978,http://www.zillow.com/homes/912087_zpid/,http://www.zillow.com/local-info/AL-Mount-Oliv...,76,AL,530 Gary Dr,"146,600",35117,912087
4,MOUNT OLIVE,http://www.zillow.com/homes/comps/912172_zpid/,http://www.zillow.com/mount-olive-al/,http://www.zillow.com/mount-olive-al/fsbo/,https://www.zillow.com/homedetails/5200-Suther...,10/25/2018,33.659778,-86.85302,http://www.zillow.com/homes/912172_zpid/,http://www.zillow.com/local-info/AL-Mount-Oliv...,70,AL,5200 Sutherland Rd,"146,600",35117,912172
5,MOUNT OLIVE,http://www.zillow.com/homes/comps/912336_zpid/,http://www.zillow.com/mount-olive-al/,http://www.zillow.com/mount-olive-al/fsbo/,https://www.zillow.com/homedetails/5260-Memory...,10/25/2018,33.658249,-86.847047,http://www.zillow.com/homes/912336_zpid/,http://www.zillow.com/local-info/AL-Mount-Oliv...,73,AL,5260 Memory Ln,"146,600",35117,912336
6,MOUNT OLIVE,http://www.zillow.com/homes/comps/912337_zpid/,http://www.zillow.com/mount-olive-al/,http://www.zillow.com/mount-olive-al/fsbo/,https://www.zillow.com/homedetails/5128-Biddle...,10/25/2018,33.658011,-86.846829,http://www.zillow.com/homes/912337_zpid/,http://www.zillow.com/local-info/AL-Mount-Oliv...,73,AL,5128 Biddle Cir,"146,600",35117,912337
7,GARDENDALE,http://www.zillow.com/homes/comps/913818_zpid/,http://www.zillow.com/gardendale-al/,http://www.zillow.com/gardendale-al/fsbo/,https://www.zillow.com/homedetails/4565-Magnol...,10/25/2018,33.639608,-86.817746,http://www.zillow.com/homes/913818_zpid/,http://www.zillow.com/local-info/AL-Gardendale...,78,AL,4565 Magnolia Ridge Dr,"150,600",35071,913818
8,GARDENDALE,http://www.zillow.com/homes/comps/914008_zpid/,http://www.zillow.com/gardendale-al/,http://www.zillow.com/gardendale-al/fsbo/,https://www.zillow.com/homedetails/4883-Cornel...,10/25/2018,33.641975,-86.847436,http://www.zillow.com/homes/914008_zpid/,http://www.zillow.com/local-info/AL-Gardendale...,72,AL,4883 Cornelius Dr,"150,600",35071,914008
9,GARDENDALE,http://www.zillow.com/homes/comps/914047_zpid/,http://www.zillow.com/gardendale-al/,http://www.zillow.com/gardendale-al/fsbo/,https://www.zillow.com/homedetails/775-Cluster...,10/25/2018,33.639801,-86.836443,http://www.zillow.com/homes/914047_zpid/,http://www.zillow.com/local-info/AL-Gardendale...,92,AL,775 Cluster Springs Rd,"150,600",35071,914047
